In [20]:
import os 
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
from os import listdir
from tqdm import tqdm
import shutil
import cv2
%matplotlib inline



In [4]:
ORIG_INPUT_DATASET = "baris/O/"
BASE_PATH = "dataset"

# Load in the images
instances = []
for filepath in os.listdir(ORIG_INPUT_DATASET):
    instances.append(cv2.imread("baris/O/{}".format(filepath)))



### Extracting only faces of objects using cascade classifier 

In [5]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
count = 0
path = 'baris/croped'
for i in instances:
    gray = cv2.cvtColor(i, cv2.COLOR_BGR2GRAY)

    face = face_cascade.detectMultiScale(gray, scaleFactor=1.5, minNeighbors=5)

    for (x,y,w,h) in face:
        cv2.rectangle(face, (x,y), (x+w,y+h), (255,0,0), 2)     
        count += 1
        # Save the captured image into the datasets folder
        cv2.imwrite(str(path) +'/' + str(count) + ".jpg", i[y:y+h,x:x+w])


        

In [6]:
count = 0
face_path = 'baris/croped'
resized_IMG_path = 'baris/resized/'
for img  in os.listdir(face_path):
    count += 1
    file = Image.open("baris/croped/{}".format(img))
    img_inter = file.resize((250, 250))
    img_inter.save('baris/resized/{}.jpg'.format(count))


In [7]:
TRAIN = "training"
TEST = "evaluation"
VAL = "validation"

BASE_PATH = "dataset"
BATCH_SIZE = 32
CLASSES = ["0", "1"] # "Neutral", 

I labeled 246 images as target values manually and saved them in labeles.txt file. 
Note: I want to detect Baris Ozcan's (a famous Youtuber) face while he is talking in his videos.

In [8]:
with open("labels.txt", 'r') as f:
    manual_labels = f.read()

manual_labels = manual_labels.replace(',', '')    
labels = [i for i in manual_labels]
len(labels)

246

In [9]:
# Important to sort the files in the order in which we see them in folder
directory = "data"
files = os.listdir(directory)
files.sort(key=lambda f: int(f.split('.')[0]))

In [10]:
trainX, testX, trainY, testY =  train_test_split(files[:len(labels)], labels, stratify=labels, train_size=0.9)

In [11]:
# further splitting of train set into train and val sets
trainX, valX, trainY, valY = train_test_split(trainX, trainY, stratify=trainY, train_size=0.85)

In [12]:
len(trainX), len(trainY), len(valX), len(valY),  len(testX), len(testY)

(187, 187, 34, 34, 25, 25)

In [15]:
# Building the dataset properly - 
splits = [(trainX, trainY), (testX, testY), (valX, valY)]
dirnames = ['training', 'evaluation', 'validation']

for i, (data,label) in enumerate(splits):
    outside_dir=dirnames[i]

    for j in tqdm(range(0, len(label)), desc="Iterating over images in sub folder"):
        dir = label[j]
        
        # construct the path to the sub-directory
        dirPath = os.path.join(BASE_PATH, outside_dir, dir)
        
        # if the output directory does not exist, create it
        if not os.path.exists(dirPath):
            os.makedirs(dirPath)
            
            
        # copy the img to this new directory
        src_img = os.path.join("data", data[j])
        shutil.copy(src_img, dirPath)

Iterating over images in sub folder: 100%|██████████| 34/34 [00:00<00:00, 54.67it/s]


In [16]:
# derive the paths to the training, validation, and testing directories
trainPath = os.path.sep.join([BASE_PATH, TRAIN])
valPath = os.path.sep.join([BASE_PATH, VAL])
testPath = os.path.sep.join([BASE_PATH, TEST])

In [18]:
# determine the total number of image paths in training, validation,
# and testing directories
from imutils import paths
totalTrain = len(list(paths.list_images(trainPath)))
totalVal = len(list(paths.list_images(valPath)))
totalTest = len(list(paths.list_images(testPath)))

In [19]:
print(totalTrain, totalTest, totalVal)

187 25 34


In [21]:
# initialize the training data augmentation object
trainAug = ImageDataGenerator(
	rotation_range=90,
	zoom_range=[0.5, 1.0],
	width_shift_range=0.3,
	height_shift_range=0.25,
	shear_range=0.15,
	horizontal_flip=True,
	fill_mode="nearest",
 	brightness_range=[0.2, 1.0]
   )

In [22]:
# Default for all the above parameters is 0, 
# meaning we are applying no augmentation to val set
# which is exactly what we need because val set should be treated like test set.
valAug = ImageDataGenerator()

In [23]:
testAug = ImageDataGenerator()

In [25]:
# Create batches whilst creating augmented images on the fly

trainGen = trainAug.flow_from_directory(
    directory=trainPath,
    target_size=(250,250),
    save_to_dir='dataset/augmented/train',
    save_prefix='train',
    shuffle=True # data will be shuffled between epochs
)

Found 187 images belonging to 2 classes.


In [26]:
valGen = valAug.flow_from_directory(
    directory=valPath,
    target_size=(250,250),
    shuffle=True
)

Found 34 images belonging to 2 classes.


In [27]:
testGen = testAug.flow_from_directory(
    directory=testPath,
    target_size=(250,250),
    shuffle=False
)

Found 25 images belonging to 2 classes.


In [28]:
img_height, img_width = 250, 250

base_model = ResNet50(weights = 'imagenet',
                      include_top = False, 
                      input_shape = (img_height, img_width, 3))


In [29]:
model_without_top = ResNet50(weights = 'imagenet',
                             include_top = False, 
                             input_shape = (img_height, img_width, 3))

                              
model_with_top = ResNet50(weights= 'imagenet',
                          include_top = True, 
                          input_shape = (224, 224, 3))



In [ ]:
model_without_top.summary()

In [ ]:
model_with_top.summary()

In [30]:
classes = ['0', '1']
num_classes = len(classes)

for layer in base_model.layers:
    layer.trainable = False

global_average_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation = 'sigmoid')(global_average_pooling)

face_classifier = keras.models.Model(inputs = base_model.input, outputs = output, name = 'ResNet50')

# saving classifier 


In [31]:
# ModelCheckpoint to save model in case of interrupting the learning process
checkpoint = ModelCheckpoint("face_classifier.h5",
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

# EarlyStopping to find best model with a large number of epochs
earlystop = EarlyStopping(monitor='val_loss',
                          restore_best_weights=True,
                          patience=3,  # number of epochs with no improvement after which training will be stopped
                          verbose=1)

callbacks = [earlystop, checkpoint]

In [32]:
face_classifier.compile(loss='categorical_crossentropy',
                        optimizer=keras.optimizers.Adam(learning_rate=0.01),
                       metrics=[tf.keras.metrics.AUC()])

In [33]:
# Training the model
hist = face_classifier.fit(
    x=trainGen,
    epochs = 50,
    verbose=2,
    validation_data=valGen,
    # shuffle=True, # argument is ignored when `x` is a generator
    steps_per_epoch=totalTrain // BATCH_SIZE,
    callbacks = [earlystop, checkpoint]
)

Epoch 1/50

Epoch 00001: val_loss improved from inf to 2.27330, saving model to face_classifier.h5


C:\Users\seyda\anaconda3\envs\kt\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


5/5 - 139s - loss: 1.6692 - auc: 0.7602 - val_loss: 2.2733 - val_auc: 0.8374 - 139s/epoch - 28s/step
Epoch 2/50

Epoch 00002: val_loss improved from 2.27330 to 0.96940, saving model to face_classifier.h5
5/5 - 121s - loss: 1.0389 - auc: 0.8971 - val_loss: 0.9694 - val_auc: 0.7444 - 121s/epoch - 24s/step
Epoch 3/50

Epoch 00003: val_loss improved from 0.96940 to 0.59899, saving model to face_classifier.h5
5/5 - 125s - loss: 0.6154 - auc: 0.9056 - val_loss: 0.5990 - val_auc: 0.9459 - 125s/epoch - 25s/step
Epoch 4/50

Epoch 00004: val_loss improved from 0.59899 to 0.36213, saving model to face_classifier.h5
5/5 - 137s - loss: 0.6507 - auc: 0.9499 - val_loss: 0.3621 - val_auc: 0.9706 - 137s/epoch - 27s/step
Epoch 5/50

Epoch 00005: val_loss did not improve from 0.36213
5/5 - 122s - loss: 0.2001 - auc: 0.9876 - val_loss: 0.4170 - val_auc: 0.9650 - 122s/epoch - 24s/step
Epoch 6/50

Epoch 00006: val_loss improved from 0.36213 to 0.26009, saving model to face_classifier.h5
5/5 - 119s - loss: 0